# Crime Rate Forecaster:

## What do we consider as a crime?

<i>For the remainder of the context, we will use wikipedia as our main source: (en.wikipedia.org/wiki/Crime)</i>

In ordinary language, a crime is an unlawful act punishable by a state or other authority. From a general standpoint, the term 'crime' does not have any simple or universally accepted definition. A proposed definition for 'crime' is one that is deemed as an act which is harmful to not only to some individual but also to a community, society, or the state. 

The main 'crime' we investigate in our model and report is <b> burglary </b>. Burglary is defined as the following: breaking & entering or housebreaking. These are considered a 'property' crime involving the illegal entry into a building or other area without permission. 

With a direct quote from the Australian Bureau of Statistics (ABS): 'The proportion of households that were broken into rose from 1.8% in 2022-23 to 2.1% in 2023-2024'. Ultimately, an increase of 32,000 extra households within 1-2 years. 

Some further information shows that 75% had reported the incident to the police with 25% not being reported. We can use this information to evaluate the potential of utilising a police notification system when the alarm is set. 

Additionally, 21% of homes were broken into multiple times, indicating a lack of security within the home to deter intruders. 

## The goal: 

The goal of this notebook and report is to identify the reason why our product needs to be developed. How would this actually prevent the crime rate from increasing and train a machine learning model to predict the crime rate over the next 10 years. 

## 1. What does the prexisting data tell us?